```
PS C:\Users\Erik\Desktop\GitHub\Atmel-SAML11\Python\pydgilib\pydgilib> ..\..\..\..\..\..\Miniconda3\envs\SAML11-32\python.exe

from dgilib import DGILib
with DGILib(verbose=3) as dgilib:
    dgilib.get_build_number()

```

- Goto folder with setup.py
- run pip install -e .
- in python:

```
import pydgilib
with DGILib(verbose=3) as dgilib
    dgilib.get_build_number()
```

In [11]:
from pydgilib import DGILib

In [7]:
from pydgilib.dgilib_exceptions import *

In [3]:
from ctypes import *
from time import sleep

In [15]:
GET_STRING_SIZE = 100
NUM_INTERFACES = 10
NUM_CONFIG_IDS = 255
NUM_CALIBRATION = 255
BUFFER_SIZE = 10000000
MAX_PRINT = 100

# Interface types
INTERFACE_TIMESTAMP  = 0x00 #   0 Service interface which appends timestamps to all received events on associated interfaces.
INTERFACE_SPI        = 0x20 #  32 Communicates directly over SPI in Slave mode.
INTERFACE_USART      = 0x21 #  33 Communicates directly over USART in Slave mode.
INTERFACE_I2C        = 0x22 #  34 Communicates directly over I2C in Slave mode.
INTERFACE_GPIO       = 0x30 #  48 Monitors and controls the state of GPIO pins.
INTERFACE_POWER_DATA = 0x40 #  64 Receives data from the attached power measurement co-processors.
INTERFACE_POWER_SYNC = 0x41 #  65 Receives sync events from the attached power measurement co-processors.
INTERFACE_RESERVED   = 0xFF # 255 Special identifier used to indicate no interface.

# Circuit types
OLD_XAM = 0x00 #   0
XAM     = 0x10 #  16
PAM     = 0x11 #  17
UNKNOWN = 0xFF # 255

# Return codes
IDLE               = 0x00 #   0
RUNNING            = 0x01 #   1
DONE               = 0x02 #   2
CALIBRATING        = 0x03 #   3
INIT_FAILED        = 0x10 #  16
OVERFLOWED         = 0x11 #  17
USB_DISCONNECTED   = 0x12 #  18
CALIBRATION_FAILED = 0x20 #  32

In [16]:
# https://www.microchip.com/developmenttools/ProductDetails/ATPOWERDEBUGGER

"""This module provides Python bindings for DGILib GPIO interface."""

__license__ = "MIT"
__author__ = "Erik Wouters <ehwo(at)kth.se>"
__credits__ = "Atmel Corporation. / Rev.: Atmel-42771A-DGILib_User Guide-09/2016"
__license__ = "MIT"
__version__ = "1.6.0"
__revision__ = " $Id: dgilib_interface_gpio.py 1586 2019-02-13 15:56:25Z EWouters $ "
__docformat__ = "reStructuredText"

NUM_PINS = 4

class DGILibInterfaceGPIO(object):
    """Python bindings for DGILib GPIO interface.

    The GPIO interface consists of four lines available, which can be individually set to input or output
    through the configuration interface. This interface can only be used in Timestamp mode. Input lines are
    monitored and will trigger an entry to be added to the timestamp buffer on each change. Output lines can
    be controlled through the send data command.
    
    ## Parsing
    Each received data byte corresponds to an input pattern on the GPIO pins. If a bit is 1 it means that the
    corresponding GPIO pin is high, a 0 means a low level.
    
    ## Configuration
    The GPIO configuration controls the direction of the pins.
    Field ID Description
    Input pins 0 Setting a bit to 1 means the pin is monitored.
    Output pins 1 Setting a bit to 1 means the pin is set to output and can be controlled by the send
    command.
    """

    def __init__(self, *args, **kwargs):
        """
        """
        
        # Argument parsing
#         self.get_config()
#         set_config
#         read_data(all pins or pin number)
        
#         self.interfaces   = kwargs.get(interfaces,[])  # List of interfaces (ints)
#         self.interface_configs = {}
#         if INTERFACE_GPIO in self.interfaces:
#             self.interface_configs[INTERFACE_GPIO]   = kwargs.get(interface_configs,[])  # List of interfaces (ints)
#         self.file_name    = kwargs.get(file_name,None)  # File name (passed to open), if empty no logging will be done.
#         self.data_in_obj  = kwargs.get(data_in_obj,False)  # if this is true the data will be available through self.timestamp[INTERFACE] and self.data[INTERFACE]
        
        
    def gpio_get_config(self):
        """Get the pin-mode for the GPIO pins
        
        The GPIO configuration controls the direction of the pins.
        
        Input pins:  Setting a bit to 1 means the pin is monitored.
        Output pins: Setting a bit to 1 means the pin is set to output and can be controlled by the send
        command.
        
        :return: Tuple of:
            - List of read modes, Setting a pin to True means the pin is monitored.
            - List of write modes, Setting a pin to True means the pin is set to output and can be controlled by the send command.
        :rtype: (list(bool), list(bool))
        """
        
        config_id, config_value = self.interface_get_configuration(INTERFACE_GPIO)
        
        read_mode = [bit is '1' for bit in f'{config_value[0]:08b}'][NUM_PINS:]
        write_mode = [bit is '1' for bit in f'{config_value[1]:08b}'][NUM_PINS:]
        
        return read_mode, write_mode
    
    def gpio_set_config(self, **kwargs):
        """Set the pin-mode for the GPIO pins
        
        The GPIO configuration controls the direction of the pins.
        
        Input pins:  Setting a bit to 1 means the pin is monitored.
        Output pins: Setting a bit to 1 means the pin is set to output and can be controlled by the send
        command.
        
        :param read_mode: List of modes, Setting a pin to True means the pin is monitored.
        :type read_mode: list(bool)
        :param write_mode: List of modes, Setting a pin to True means the pin is set to output and can be controlled by the send command.
        :type write_mode: list(bool)
        """
        
        # Get optional arguments
        read_mode = kwargs.get("read_mode",[False] * NUM_PINS)
        write_mode = kwargs.get("write_mode",[False] * NUM_PINS)
        
        # convert lists of bool to int
        read_mode = int(''.join('1' if i else '0' for i in read_mode), 2)
        write_mode = int(''.join('1' if i else '0' for i in write_mode), 2)
        
        if 'read_mode' in kwargs:
            self.interface_set_configuration(INTERFACE_GPIO, [0], [read_mode])
        if 'write_mode' in kwargs:
            self.interface_set_configuration(INTERFACE_GPIO, [1], [write_mode])
    
    def gpio_read(self, **kwargs):
        """Get the state of the GPIO pins
        
        :param read_mode: List of modes, Setting a pin to True means the pin is monitored.
        :type read_mode: list(bool)
        :param write_mode: List of modes, Setting a pin to True means the pin is set to output and can be controlled by the send command.
        :type write_mode: list(bool)
        """
        
        
        
        return self.interface_read_data(INTERFACE_GPIO)
    
    def gpio_write(self, **kwargs):
        """Set the state of the GPIO pins
        
        :param read_mode: List of modes, Setting a pin to True means the pin is monitored.
        :type read_mode: list(bool)
        :param write_mode: List of modes, Setting a pin to True means the pin is set to output and can be controlled by the send command.
        :type write_mode: list(bool)
        """
        
        # Get optional arguments
        read_mode = kwargs.get("read_mode",[False] * NUM_PINS)
        write_mode = kwargs.get("write_mode",[False] * NUM_PINS)
        
        # convert lists of bool to int
        read_mode = int(''.join('1' if i else '0' for i in read_mode), 2)
        write_mode = int(''.join('1' if i else '0' for i in write_mode), 2)
        
        if 'read_mode' in kwargs:
            self.interface_set_configuration(INTERFACE_GPIO, [0], [read_mode])
        if 'write_mode' in kwargs:
            self.interface_set_configuration(INTERFACE_GPIO, [1], [write_mode])

# config_value[0] = 0b1111
# config_value[1] = 0b0000
# res = dgilib.interface_set_configuration(dgi_hndl, interface_id, byref(config_id), byref(config_value), config_cnt)
# print("{0} interface_set_configuration: {1}, config_cnt: {2}".format(res, interface_id, repr(config_cnt.value)))
# for i in range(config_cnt.value):
#     print("\tconfig_id: {0},\tvalue: {1}".format(config_id[i], config_value[i]))
    
#         self.connect()
#         c_status = self.connection_status()
#         if c_status:
#             raise DeviceConnectionError(
#                 f"Could not connect to device. Connection status: {c_status}."
#             )
#         self.interface_list()
#         self.interface_enable(None, INTERFACE_GPIO, True)

#         self.start_polling()

#         self.stop_polling()
#         self.interface_disable(None, INTERFACE_GPIO)
#         self.disconnect()

In [17]:
# https://www.microchip.com/developmenttools/ProductDetails/ATPOWERDEBUGGER

"""This module provides Python bindings for DGILib Logger."""

__license__ = "MIT"
__author__ = "Erik Wouters <ehwo(at)kth.se>"
__credits__ = "Atmel Corporation. / Rev.: Atmel-42771A-DGILib_User Guide-09/2016"
__license__ = "MIT"
__version__ = "1.6.0"
__revision__ = " $Id: dgilib_logger.py 1586 2019-02-13 15:56:25Z EWouters $ "
__docformat__ = "reStructuredText"

import csv

class DGILibLogger(object):
    """Python bindings for DGILib Logger.
    """

    def __init__(self, *args, **kwargs):
        """
        """
        
        # Argument parsing
#         interfaces
#             - GPIO mode
#             - Power mode
        
#         file_name
#         plot
        
#         update_callback
        
        self.interfaces   = kwargs.get("interfaces",[])  # List of interfaces (ints)
#         self.interface_configs = {}
#         if INTERFACE_GPIO in self.interfaces:
#             self.interface_configs[INTERFACE_GPIO]   = kwargs.get("interface_configs",[])  # List of interfaces (ints)
        self.file_name    = kwargs.get("file_name",None)  # File name (passed to open), if empty no logging will be done.
        self.data_in_obj  = kwargs.get("data_in_obj",False)  # if this is true the data will be available through self.timestamp[INTERFACE] and self.data[INTERFACE]
        

#         output = csv.writer(open('export_log.csv', 'w'))

#         for foo in bar:
#            # do work
#            output.writerow([status, view, filename, out, err, current_time])
        
        
#         self.options = {
#             'option1' : 'default_value1',
#             'option2' : 'default_value2',
#             'option3' : 'default_value3', }

#         self.options.update(kwargs)
#         print self.options
        
    def __enter__(self):
        """
        """
        
        if self.file_name is not None:
            # TODO per interface
            self.writer = "csv.writer"(open(file_name, 'w'))
        
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        """
        """
        
            

    def logger(self, duration=10):
        """logger
        """
        
        
#         self.writer[INTERFACE_GPIO].writerow(['status', 'view', 'filename', 'stdout', 'stderr', 'time'])

        dgilib.auxiliary_power_get_status()
        dgilib.auxiliary_power_get_calibration()
        dgilib.auxiliary_power_register_buffer_pointers()
        dgilib.start_polling()
        dgilib.auxiliary_power_start()
        dgilib.auxiliary_power_get_status()
        sleep(duration)
        dgilib.auxiliary_power_lock_data_for_reading()
        dat = dgilib.auxiliary_power_copy_data()
        dgilib.auxiliary_power_free_data()
        dgilib.auxiliary_power_stop()
        dgilib.auxiliary_power_get_status()

        dgilib.auxiliary_power_unregister_buffer_pointers()
        #     dgilib.auxiliary_power_uninitialize()

        dgilib.stop_polling()

        return dat
    
#     int loggedSamples = 0;
#     printf("Waiting to log some data \n");
#     for (int i = 0; i < 25; i++) {
#         auxiliary_power_lock_data_for_reading(power_hndl);
#         auxiliary_power_copy_data(power_hndl, powerBuffer+loggedSamples, powerTimestamp+loggedSamples, &powerCount, max_count, channel, powerType);
#         auxiliary_power_free_data(power_hndl);
#         loggedSamples += powerCount;
#         printf(".%d", powerCount);
# //        printf(".");
#         for (int j = 0;j < WAIT_ITERATIONS; j+=10) {}
#     }
#     printf(" Done\n");

In [18]:
# https://www.microchip.com/developmenttools/ProductDetails/ATPOWERDEBUGGER

"""This module provides Python bindings for DGILib Extra."""

__license__ = "MIT"
__author__ = "Erik Wouters <ehwo(at)kth.se>"
__credits__ = "Atmel Corporation. / Rev.: Atmel-42771A-DGILib_User Guide-09/2016"
__license__ = "MIT"
__version__ = "1.6.0"
__revision__ = " $Id: dgilib_extra.py 1586 2019-02-13 15:56:25Z EWouters $ "
__docformat__ = "reStructuredText"

# from dgilib_interface_gpio import DGILibInterfaceGPIO
# from dgilib_extra import DGILibLogger

class DGILibExtra(DGILib, DGILibInterfaceGPIO, DGILibLogger):
    """Python bindings for DGILib Extra.
    """

    def __init__(
        self, *args, **kwargs
    ):
        """
        """

        DGILib.__init__(self, *args, **kwargs)
#         DGILibInterfaceGPIO.__init__(self, *args, **kwargs)
        DGILibLogger.__init__(self, *args, **kwargs)

    def __enter__(self):
        """
        """

        DGILib.__enter__(self)
#         DGILibInterfaceGPIO.__enter__(self)
        DGILibLogger.__enter__(self)

        return self

    def __exit__(self, exc_type, exc_value, traceback):

        DGILibLogger.__exit__(self, exc_type, exc_value, traceback)
#         DGILibInterfaceGPIO.__exit__(self, exc_type, exc_value, traceback)
        DGILib.__exit__(self, exc_type, exc_value, traceback)
        
        if self.verbose:
            print("bye from DGILib Extra")

    def info(self):
        """Get the build information of DGILib.
        
        :param print_info: A flag used to print the build information to the console (default is False)
        :type print_info: bool
        :return:  Version information of DGILib:
            - major_version: the major_version of DGILib
            - minor_version: the minor_version of DGILib
            - build_number: the build number of DGILib. 0 if not supported
            - major_fw: the major firmware version of the DGI device connected
            - minor_fw: the minor firmware version of the DGI device connected
        :rtype: tuple
        """

        major_version = self.get_major_version()
        minor_version = self.get_minor_version()
        build_number = self.get_build_number()
        major_fw, minor_fw = self.get_fw_version()

        return (major_version, minor_version, build_number, major_fw, minor_fw)

    def device_reset(self, duration=1):
        """Set the device reset line for duration seconds.
        """

        dgilib.target_reset(True)
        sleep(duration)
        dgilib.target_reset(False)

    def get_time_factor(self):
        """Get the factor to multiply timestamps by to get seconds.
        
        :return: timer_factor
        :rtype: double
        """

        _, config_value = self.interface_get_configuration(INTERFACE_TIMESTAMP)
        timer_prescaler = config_value[0]
        timer_frequency = config_value[1]

        if self.verbose:
            print(
                f"timer_factor: {timer_prescaler / timer_frequency}, timer_prescaler: {timer_prescaler}, timer_frequency: {timer_frequency}"
            )

        return timer_prescaler / timer_frequency

    def auxiliary_power_calibration(self, circuit_type=XAM):
        """Calibrate the Auxilary Power interface of the device.
        
        :param circuit_type: Type of calibration to trigger (defaults to XAM)
        :type circuit_type: int
        """

        self.auxiliary_power_trigger_calibration(circuit_type)
        while self.auxiliary_power_get_status() == CALIBRATING:
            sleep(0.1)

In [19]:
with DGILibExtra(verbose=0) as dgilib:
    config = dgilib.gpio_get_config()
    print(config)
    dgilib.gpio_set_config(read_mode=[True, True, False, False], write_mode=[True, False, True, False])
    config = dgilib.gpio_get_config()
    print(config)
    gpio_data = dgilib.gpio_read()
    print(gpio_data)

([False, False, False, False], [False, False, False, False])
([True, True, False, False], [True, False, True, False])
([], [])


In [13]:
with DGILibExtra(verbose=1) as dgilib:
    dgilib.get_time_factor()
    dgilib.auxiliary_power_calibration()

device_count: 1
	0 get_device_serial: b'ATML3138061800001604'
	0 connect
connection_status: 0
	0 auxiliary_power_initialize
	0 interface_get_configuration: 0, config_cnt: 2
timer_factor: 5.333333333333333e-07, timer_prescaler: 32, timer_frequency: 60000000
	0 auxiliary_power_trigger_calibration
power_status: 3
power_status: 2
	0 auxiliary_power_uninitialize
bye from Auxiliary
	0 disconnect
bye from Housekeeping
bye from Discovery
bye from DGILib
bye from DGILib Extra


In [14]:
dat = []
with DGILibExtra(verbose=0) as dgilib:
    dat = dgilib.logger()

In [15]:
%matplotlib ipympl

In [16]:
import matplotlib.pyplot as plt
import numpy as np

In [17]:
plt.plot(dat[1], dat[0])
plt.show()

C:\Users\Erik\Miniconda3\envs\SAML11-32\lib\site-packages\matplotlib\__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

In [22]:
import distutils
help(distutils)

Help on package distutils:

NAME
    distutils - distutils

DESCRIPTION
    The main package for the Python Module Distribution Utilities.  Normally
    used from a setup script as
    
       from distutils.core import setup
    
       setup (...)

PACKAGE CONTENTS
    _msvccompiler
    archive_util
    bcppcompiler
    ccompiler
    cmd
    command (package)
    config
    core
    cygwinccompiler
    debug
    dep_util
    dir_util
    dist
    errors
    extension
    fancy_getopt
    file_util
    filelist
    log
    msvc9compiler
    msvccompiler
    spawn
    sysconfig
    tests (package)
    text_file
    unixccompiler
    util
    version
    versionpredicate

VERSION
    3.6.5

FILE
    c:\users\erik\miniconda3\envs\saml11-32\lib\distutils\__init__.py


